In [ ]:
import matplotlib.pyplot as plt
from scipy.constants import sigma
from scipy.optimize import minimize
import os
import pandas as pd
import cvxpy as cp
import numpy as np

from level1.functions import f_returns_on_df, f_mu_on_df, f_sigma_on_df
from portfolio_utils import load_datas, f_yield, f_volatility
from level3.functions import boostrap_sample_df
from level2.functions import optimize

# Charger les données
df = load_datas()

In [ ]:
df.info()

In [ ]:
set(df.index.year)

In [ ]:
c = 0.001
K = 2
re = f_returns_on_df(df)
mu = f_mu_on_df(re)
Sigma = f_sigma_on_df(re)
num_assets = len(mu)
w0 = np.array(len(mu) * [0])
w0[2] = 1

In [ ]:
frontier_yields, frontier_volatility, frontier_cost, frontier_weights = optimize(mu, Sigma, w0, K, delta_tol=0.01, population_size=300, generations=100, c=c)

In [ ]:
year_params = {}
for year, b in boostrap_sample_df(df).items():
    re_y = f_returns_on_df(b)
    mu_y = f_mu_on_df(re_y)
    Sigma_y = f_sigma_on_df(re_y)
    year_params[year] = (mu_y, Sigma_y)


In [ ]:
def evaluate_portfolio_over_years(w, year_params):
    rets = []
    vols = []
    for mu_y, Sigma_y in year_params.values():
        r = f_yield(w, mu_y)
        v = f_volatility(w, Sigma_y)
        rets.append(r)
        vols.append(v)
    return np.array(rets), np.array(vols)

std_yields = []
std_vols = []

for w in frontier_weights:
    rets, vols = evaluate_portfolio_over_years(w, year_params)
    std_yields.append(np.std(rets))   # f4 = instabilité rendement
    std_vols.append(np.std(vols))


In [ ]:
def normalize(x):
    x = np.array(x)
    return (x - x.min()) / (x.max() - x.min() + 1e-12) # 1e-12 to avoid zero division

# On normalise pour avoir la meme echelle

score = (
    0.4 * normalize(frontier_volatility) +   # niveau de risque
    0.1 * normalize(frontier_cost) +          # coûts
    0.4 * normalize(std_yields) +             # instabilité rendement
    0.1 * normalize(std_vols)                 # instabilité risque
)

In [ ]:
best_idx = np.argmin(score)
w_robust = frontier_weights[best_idx]
print(f"Portefeuille robuste sélectionné :")
print(f"Rendement : {frontier_yields[best_idx]:.4f}")
print(f"Volatilité : {frontier_volatility[best_idx]:.4f}")
print(f"Coût : {frontier_cost[best_idx]:.4f}")
print(f"Instabilité rendement (std) : {std_yields[best_idx]:.4f}")
for i, weight in enumerate(w_robust):
    if weight > 1e-4:
        print(f"  {df.columns[i]} : {weight:.4f}")